## Goal of this notebook
We know from early notebooks that three day momentum models offer the greatest predictive power, then the two day model, then the one day (base) model.

However, how can we be sure of the accuracy of these models, or that they generalize well enough?

In this notebook, we will test these models against each other on random subsets of data, to see which model is able to predict the data most accurately. Our hope, of course, is that the three day model will win out, since it provides th

If there is time, and if the two day model beats the three day model, it may be worth taking the time to build an aggregation of the two, where the three day model will use the two day model for predictions when there is a sequence for which we do not have enough data points (i.e. count_sequence < some tyhreshold). Of course, this may remedy itself when we add more data.

## TODO 
 - Test one day vs. three day
 - Modularize Notebook
 - Rerun tests over all stocks

In [1]:
from stock_utils import *

In [2]:
df = pd.DataFrame()
df = df.from_csv('stock_data/nvda.csv')

daily_movements = get_price_movements(df)
movement_categories = categorize_movements(daily_movements, n_cats=4)

two_day_movement_trends = get_trends(movement_categories, 2)
three_day_movement_trends = get_trends(movement_categories, 3)

In [3]:
def get_single_day_probabilities(movement_categories):
    movement_category_types = ['bd', 'sd', 'sg', 'bg']
    single_day_counts = []
    single_day_probabilities = []
    total = 0
    
    for cat in movement_category_types:
        count = count_movement_category(movement_categories, cat)
        single_day_counts.append(count)
        total += count
    
    for count in single_day_counts:
        single_day_probabilities.append(count/total)
        
    return single_day_probabilities

In [4]:
def get_probabilities_after_event(previous_event_category, trends, movement_categories):
    """
    Given an event that occured the previous day, return the probabilities of the next day's
    movement categories conditioned on said event.
    
    Arguments:
      previous_event_category -- The category of the event we observed the previous day 
                                 (or two days in the case of three day momentum)
      trends -- All two (or three) day trends that were observed for this event type.
      movement_categories -- all daily movement categories that were observed
    
    Returns:
      next_day_movement_probabilities -- Probabilities of each of the next day's categories
                                         conditioned on the previous event category
    """
    movement_category_types = ['bd', 'sd', 'sg', 'bg']
    next_day_movement_probabilities = []
    
    for next_day in movement_category_types:
        trend_name = previous_event_category + '_' + next_day
        trend_count = count_trends(trends, trend_name)
            
        trend_total = 0
        for category in movement_category_types:
            trend_total += count_trends(trends, previous_event_category + '_' + category)
            
        trend_prob = trend_count / trend_total
        next_day_movement_probabilities.append(trend_prob)
        
    return next_day_movement_probabilities

## Test localized two-day momentum model against the localized one day model
One day model blows the two day model out of the water. It seems there is not enough data for the two day model to generalize well.

In [5]:
index2category = {0:'bd', 1:'sd', 2:'sg', 3:'bg'}
category2index = {'bd':0, 'sd':1, 'sg':2, 'bg':3}

In [6]:
def select_data_sample(data, sample_size):
    ## We are going to use the next sample_size elements, so it can't be at the end where we won't have enough elements
    sub_sample = data[0:-sample_size]
    random_index = random.choice(list(enumerate(sub_sample)))[0]
    return data[random_index:random_index+sample_size]

In [7]:
single_day_probs = get_single_day_probabilities(movement_categories)
all_two_day_probs = []
all_one_day_probs = []
movement_category_types = ['bd', 'sd', 'sg', 'bg']
for cat in movement_category_types:
    all_two_day_probs.append(get_probabilities_after_event(cat, two_day_movement_trends, movement_categories))
    all_one_day_probs.append(single_day_probs) ## I want them to be the same dimension

In [8]:
n_tries = 10000
one_wins = 0
two_wins = 0
for a in range(n_tries):
    sample = select_data_sample(movement_categories, 50)
    sample_data_probabilities = get_single_day_probabilities(sample)
    
    one_count = 0
    two_count = 0

    for i in range(len(sample) - 1):
        prev_day = sample[i]
        next_day = sample[i+1]
    
        one_day_model_next_day_probs = all_one_day_probs[category2index[prev_day]]
        two_day_model_next_day_probs = all_two_day_probs[category2index[prev_day]]
    
        ## Weight correct answers on larger movements more heavily
        if(next_day == 'bg' or next_day == 'bd'):
            if (one_day_model_next_day_probs[category2index[next_day]] > two_day_model_next_day_probs[category2index[next_day]]):
                one_count += 2
            else:
                two_count += 2
        else:
            if (one_day_model_next_day_probs[category2index[next_day]] > two_day_model_next_day_probs[category2index[next_day]]):
                one_count += 1
            else:
                two_count += 1

    if one_count > two_count:
        one_wins += 1
    else:
        two_wins += 1
    #print('one count is: ' + str(one_count) + ' and two_count is ' + str(two_count))
    
print('One day model won ' + str(one_wins) + ' times')
print('Two day model won ' + str(two_wins) + ' times')

One day model won 8958 times
Two day model won 1042 times


## Test composite two day momentum model against the composite one day model
What I mean by "composite" is that it is created by averaging over all the stocks in our stock_data folder.

In this case, the two day model blows the one day model out of the water, which we should expect, given that the two day model provides us with added context and information.

In [9]:
all_two_day_trends, all_two_day_category_counts, all_single_day_category_probs, all_movement_categories = \
    get_trends_all_stocks(1, 2, movement_category_types, n_cats=4)

In [10]:
all_two_day_probs = []
all_one_day_probs = []
movement_category_types = ['bd', 'sd', 'sg', 'bg']
for cat in movement_category_types:
    all_two_day_probs.append(get_probabilities_after_event(cat, all_two_day_trends, all_movement_categories))
    all_one_day_probs.append(all_single_day_category_probs) ## I want them to be the same dimension

In [11]:
n_tries = 10000
one_wins = 0
two_wins = 0
for a in range(n_tries):
    sample = select_data_sample(all_movement_categories, 50)
    sample_data_probabilities = get_single_day_probabilities(sample)
    
    one_count = 0
    two_count = 0

    for i in range(len(sample) - 1):
        prev_day = sample[i]
        next_day = sample[i+1]
    
        one_day_model_next_day_probs = all_one_day_probs[category2index[prev_day]]
        two_day_model_next_day_probs = all_two_day_probs[category2index[prev_day]]
    
        ## Weight correct answers on larger movements more heavily
        if(next_day == 'bg' or next_day == 'bd'):
            if (one_day_model_next_day_probs[category2index[next_day]] > two_day_model_next_day_probs[category2index[next_day]]):
                one_count += 2
            else:
                two_count += 2
        else:
            if (one_day_model_next_day_probs[category2index[next_day]] > two_day_model_next_day_probs[category2index[next_day]]):
                one_count += 1
            else:
                two_count += 1

    if one_count > two_count:
        one_wins += 1
    else:
        two_wins += 1
    #print('one count is: ' + str(one_count) + ' and two_count is ' + str(two_count))
    
print('One day model won ' + str(one_wins) + ' times')
print('Two day model won ' + str(two_wins) + ' times')

One day model won 1444 times
Two day model won 8556 times


## Test composite two day momentum model against the localized one day model
Again, the generalized two-day model wins by a large margin.

In [12]:
single_day_probs = get_single_day_probabilities(movement_categories)
all_two_day_probs = []
all_one_day_probs = []
movement_category_types = ['bd', 'sd', 'sg', 'bg']
for cat in movement_category_types:
    all_two_day_probs.append(get_probabilities_after_event(cat, all_two_day_trends, all_movement_categories))
    all_one_day_probs.append(single_day_probs) ## I want them to be the same dimension

In [13]:
n_tries = 10000
one_wins = 0
two_wins = 0
for a in range(n_tries):
    sample = select_data_sample(all_movement_categories, 50)
    sample_data_probabilities = get_single_day_probabilities(sample)
    
    one_count = 0
    two_count = 0

    for i in range(len(sample) - 1):
        prev_day = sample[i]
        next_day = sample[i+1]
    
        one_day_model_next_day_probs = all_one_day_probs[category2index[prev_day]]
        two_day_model_next_day_probs = all_two_day_probs[category2index[prev_day]]
    
        ## Weight correct answers on larger movements more heavily
        if(next_day == 'bg' or next_day == 'bd'):
            if (one_day_model_next_day_probs[category2index[next_day]] > two_day_model_next_day_probs[category2index[next_day]]):
                one_count += 2
            else:
                two_count += 2
        else:
            if (one_day_model_next_day_probs[category2index[next_day]] > two_day_model_next_day_probs[category2index[next_day]]):
                one_count += 1
            else:
                two_count += 1

    if one_count > two_count:
        one_wins += 1
    else:
        two_wins += 1
    #print('one count is: ' + str(one_count) + ' and two_count is ' + str(two_count))
    
print('One day model won ' + str(one_wins) + ' times')
print('Two day model won ' + str(two_wins) + ' times')

One day model won 1375 times
Two day model won 8625 times


#### There is no need to even try the localized three day model, since it is pretty obvious that it will generalize poorly

## Test composite three day momentum model against the generalized one day model
A perfect victory for the three day model. That's incredible.

In [16]:
all_three_day_trends, all_single_day_category_counts, all_single_day_category_probs, all_movement_categories = \
    get_trends_all_stocks(1, 3, movement_category_types, n_cats=4)

In [22]:
all_three_day_probs = []
all_one_day_probs = []
movement_category_types = ['bd', 'sd', 'sg', 'bg']
for cat in movement_category_types:
    for cat2 in movement_category_types:
        #print(cat + '_' + cat2)
        all_three_day_probs.append(get_probabilities_after_event(cat + '_' + cat2, all_three_day_trends, all_movement_categories))
        all_one_day_probs.append(all_single_day_category_probs) ## I want them to be the same dimension

In [30]:
n_tries = 10000
one_wins = 0
three_wins = 0
for a in range(n_tries):
    sample = select_data_sample(all_movement_categories, 50)
    sample_data_probabilities = get_single_day_probabilities(sample)
    
    one_count = 0
    three_count = 0

    for i in range(len(sample) - 2):
        day_before_last = sample[i]
        prev_day = sample[i+1]
        next_day = sample[i+2]
    
        one_day_model_next_day_probs = all_one_day_probs[0]
        three_day_model_next_day_probs = all_three_day_probs[category2index[day_before_last] * 4 + category2index[prev_day]]
        
        ##print(next_day)
        ##print('Probability given by the one day model is ' + str(one_day_model_next_day_probs[category2index[next_day]]))
        ##print('Probability given by the three day model is ' + str(three_day_model_next_day_probs[category2index[next_day]]))
    
        ## Weight correct answers on larger movements more heavily
        if(next_day == 'bg' or next_day == 'bd'):
            if (one_day_model_next_day_probs[category2index[next_day]] > three_day_model_next_day_probs[category2index[next_day]]):
                one_count += 2
            else:
                three_count += 2
        else:
            if (one_day_model_next_day_probs[category2index[next_day]] > two_day_model_next_day_probs[category2index[next_day]]):
                one_count += 1
            else:
                three_count += 1

    if one_count > two_count:
        one_wins += 1
    else:
        three_wins += 1
    #print('one count is: ' + str(one_count) + ' and three_count is ' + str(three_count))
    
print('One day model won ' + str(one_wins) + ' times')
print('Three day model won ' + str(three_wins) + ' times')

one count is: 18 and three_count is 41
one count is: 8 and three_count is 52
one count is: 16 and three_count is 48
one count is: 8 and three_count is 45
one count is: 10 and three_count is 44
one count is: 12 and three_count is 50
one count is: 16 and three_count is 51
one count is: 16 and three_count is 56
one count is: 4 and three_count is 48
one count is: 12 and three_count is 48
one count is: 6 and three_count is 73
one count is: 12 and three_count is 66
one count is: 4 and three_count is 48
one count is: 4 and three_count is 46
one count is: 8 and three_count is 44
one count is: 8 and three_count is 51
one count is: 12 and three_count is 52
one count is: 12 and three_count is 44
one count is: 8 and three_count is 56
one count is: 10 and three_count is 47
one count is: 8 and three_count is 50
one count is: 4 and three_count is 74
one count is: 14 and three_count is 47
one count is: 12 and three_count is 55
one count is: 10 and three_count is 47
one count is: 10 and three_count is 

one count is: 12 and three_count is 42
one count is: 10 and three_count is 45
one count is: 10 and three_count is 47
one count is: 10 and three_count is 46
one count is: 16 and three_count is 50
one count is: 10 and three_count is 49
one count is: 16 and three_count is 50
one count is: 10 and three_count is 64
one count is: 6 and three_count is 45
one count is: 12 and three_count is 46
one count is: 14 and three_count is 54
one count is: 10 and three_count is 44
one count is: 4 and three_count is 46
one count is: 16 and three_count is 46
one count is: 14 and three_count is 54
one count is: 10 and three_count is 45
one count is: 12 and three_count is 47
one count is: 14 and three_count is 52
one count is: 4 and three_count is 46
one count is: 10 and three_count is 47
one count is: 8 and three_count is 44
one count is: 8 and three_count is 47
one count is: 8 and three_count is 45
one count is: 8 and three_count is 47
one count is: 18 and three_count is 39
one count is: 6 and three_count 

one count is: 6 and three_count is 45
one count is: 10 and three_count is 50
one count is: 14 and three_count is 49
one count is: 8 and three_count is 50
one count is: 12 and three_count is 54
one count is: 14 and three_count is 43
one count is: 6 and three_count is 46
one count is: 16 and three_count is 43
one count is: 4 and three_count is 46
one count is: 6 and three_count is 46
one count is: 12 and three_count is 44
one count is: 12 and three_count is 54
one count is: 10 and three_count is 47
one count is: 12 and three_count is 43
one count is: 4 and three_count is 46
one count is: 8 and three_count is 49
one count is: 4 and three_count is 49
one count is: 16 and three_count is 44
one count is: 6 and three_count is 47
one count is: 12 and three_count is 57
one count is: 10 and three_count is 56
one count is: 0 and three_count is 90
one count is: 6 and three_count is 45
one count is: 2 and three_count is 86
one count is: 6 and three_count is 49
one count is: 8 and three_count is 45


one count is: 10 and three_count is 44
one count is: 4 and three_count is 49
one count is: 10 and three_count is 44
one count is: 2 and three_count is 49
one count is: 4 and three_count is 48
one count is: 4 and three_count is 47
one count is: 8 and three_count is 46
one count is: 2 and three_count is 48
one count is: 14 and three_count is 55
one count is: 6 and three_count is 59
one count is: 4 and three_count is 46
one count is: 2 and three_count is 82
one count is: 12 and three_count is 51
one count is: 14 and three_count is 58
one count is: 10 and three_count is 54
one count is: 14 and three_count is 41
one count is: 12 and three_count is 48
one count is: 2 and three_count is 49
one count is: 8 and three_count is 44
one count is: 12 and three_count is 45
one count is: 10 and three_count is 46
one count is: 12 and three_count is 51
one count is: 10 and three_count is 45
one count is: 18 and three_count is 45
one count is: 8 and three_count is 47
one count is: 16 and three_count is 4

one count is: 6 and three_count is 46
one count is: 4 and three_count is 46
one count is: 6 and three_count is 46
one count is: 6 and three_count is 46
one count is: 18 and three_count is 47
one count is: 8 and three_count is 44
one count is: 8 and three_count is 45
one count is: 8 and three_count is 45
one count is: 16 and three_count is 50
one count is: 18 and three_count is 46
one count is: 6 and three_count is 45
one count is: 8 and three_count is 44
one count is: 12 and three_count is 46
one count is: 6 and three_count is 46
one count is: 12 and three_count is 56
one count is: 6 and three_count is 47
one count is: 6 and three_count is 48
one count is: 16 and three_count is 44
one count is: 10 and three_count is 49
one count is: 14 and three_count is 44
one count is: 18 and three_count is 44
one count is: 12 and three_count is 57
one count is: 10 and three_count is 46
one count is: 8 and three_count is 46
one count is: 12 and three_count is 46
one count is: 4 and three_count is 46


...

one count is: 10 and three_count is 46
one count is: 4 and three_count is 46
one count is: 6 and three_count is 46
one count is: 12 and three_count is 43
one count is: 0 and three_count is 48
one count is: 14 and three_count is 54
one count is: 4 and three_count is 49
one count is: 8 and three_count is 56
one count is: 12 and three_count is 45
one count is: 12 and three_count is 46
one count is: 16 and three_count is 47
one count is: 16 and three_count is 47
one count is: 14 and three_count is 52
one count is: 4 and three_count is 49
one count is: 4 and three_count is 46
one count is: 10 and three_count is 60
one count is: 4 and three_count is 47
one count is: 4 and three_count is 46
one count is: 10 and three_count is 46
one count is: 12 and three_count is 46
one count is: 0 and three_count is 48
one count is: 8 and three_count is 45
one count is: 10 and three_count is 50
one count is: 4 and three_count is 49
one count is: 6 and three_count is 45
one count is: 8 and three_count is 48


one count is: 2 and three_count is 47
one count is: 2 and three_count is 48
one count is: 10 and three_count is 45
one count is: 12 and three_count is 44
one count is: 2 and three_count is 47
one count is: 10 and three_count is 46
one count is: 12 and three_count is 54
one count is: 10 and three_count is 47
one count is: 8 and three_count is 49
one count is: 12 and three_count is 42
one count is: 8 and three_count is 46
one count is: 8 and three_count is 44
one count is: 14 and three_count is 45
one count is: 16 and three_count is 55
one count is: 16 and three_count is 47
one count is: 6 and three_count is 45
one count is: 10 and three_count is 59
one count is: 16 and three_count is 46
one count is: 14 and three_count is 50
one count is: 16 and three_count is 46
one count is: 14 and three_count is 48
one count is: 10 and three_count is 49
one count is: 18 and three_count is 50
one count is: 10 and three_count is 43
one count is: 16 and three_count is 43
one count is: 10 and three_count

one count is: 16 and three_count is 44
one count is: 18 and three_count is 44
one count is: 4 and three_count is 47
one count is: 10 and three_count is 44
one count is: 0 and three_count is 50
one count is: 4 and three_count is 49
one count is: 10 and three_count is 44
one count is: 4 and three_count is 47
one count is: 0 and three_count is 48
one count is: 4 and three_count is 46
one count is: 4 and three_count is 48
one count is: 10 and three_count is 45
one count is: 12 and three_count is 46
one count is: 10 and three_count is 44
one count is: 6 and three_count is 45
one count is: 12 and three_count is 51
one count is: 12 and three_count is 43
one count is: 12 and three_count is 43
one count is: 2 and three_count is 84
one count is: 6 and three_count is 45
one count is: 14 and three_count is 49
one count is: 10 and three_count is 47
one count is: 12 and three_count is 50
one count is: 14 and three_count is 50
one count is: 6 and three_count is 48
one count is: 6 and three_count is 4

one count is: 8 and three_count is 44
one count is: 10 and three_count is 45
one count is: 4 and three_count is 46
one count is: 2 and three_count is 47
one count is: 6 and three_count is 45
one count is: 8 and three_count is 46
one count is: 8 and three_count is 45
one count is: 2 and three_count is 47
one count is: 12 and three_count is 45
one count is: 2 and three_count is 47
one count is: 10 and three_count is 44
one count is: 10 and three_count is 60
one count is: 6 and three_count is 49
one count is: 6 and three_count is 47
one count is: 12 and three_count is 43
one count is: 8 and three_count is 45
one count is: 12 and three_count is 54
one count is: 8 and three_count is 52
one count is: 2 and three_count is 47
one count is: 6 and three_count is 47
one count is: 10 and three_count is 46
one count is: 2 and three_count is 50
one count is: 10 and three_count is 44
one count is: 8 and three_count is 62
one count is: 10 and three_count is 44
one count is: 14 and three_count is 49
on

one count is: 6 and three_count is 45
one count is: 10 and three_count is 51
one count is: 12 and three_count is 49
one count is: 2 and three_count is 49
one count is: 8 and three_count is 44
one count is: 2 and three_count is 47
one count is: 6 and three_count is 45
one count is: 10 and three_count is 44
one count is: 8 and three_count is 45
one count is: 4 and three_count is 46
one count is: 12 and three_count is 54
one count is: 12 and three_count is 53
one count is: 16 and three_count is 48
one count is: 14 and three_count is 49
one count is: 16 and three_count is 52
one count is: 6 and three_count is 45
one count is: 4 and three_count is 47
one count is: 10 and three_count is 44
one count is: 12 and three_count is 54
one count is: 8 and three_count is 47
one count is: 10 and three_count is 54
one count is: 8 and three_count is 46
one count is: 12 and three_count is 62
one count is: 4 and three_count is 49
one count is: 8 and three_count is 44
one count is: 2 and three_count is 47


## Test composite three day momentum model against the localized one day model
Again, an absolute victory

In [32]:
df = df.from_csv('stock_data/nvda.csv')

daily_movements = get_price_movements(df)
movement_categories = categorize_movements(daily_movements, n_cats=4)
single_day_probs = get_single_day_probabilities(movement_categories)

In [33]:
all_three_day_probs = []
all_one_day_probs = []
movement_category_types = ['bd', 'sd', 'sg', 'bg']
for cat in movement_category_types:
    for cat2 in movement_category_types:
        #print(cat + '_' + cat2)
        all_three_day_probs.append(get_probabilities_after_event(cat + '_' + cat2, all_three_day_trends, all_movement_categories))
        all_one_day_probs.append(single_day_probs) ## I want them to be the same dimension

In [34]:
n_tries = 10000
one_wins = 0
three_wins = 0
for a in range(n_tries):
    sample = select_data_sample(movement_categories, 50)
    sample_data_probabilities = get_single_day_probabilities(sample)
    
    one_count = 0
    three_count = 0

    for i in range(len(sample) - 2):
        day_before_last = sample[i]
        prev_day = sample[i+1]
        next_day = sample[i+2]
    
        one_day_model_next_day_probs = all_one_day_probs[0]
        three_day_model_next_day_probs = all_three_day_probs[category2index[day_before_last] * 4 + category2index[prev_day]]
        
        ##print(next_day)
        ##print('Probability given by the one day model is ' + str(one_day_model_next_day_probs[category2index[next_day]]))
        ##print('Probability given by the three day model is ' + str(three_day_model_next_day_probs[category2index[next_day]]))
    
        ## Weight correct answers on larger movements more heavily
        if(next_day == 'bg' or next_day == 'bd'):
            if (one_day_model_next_day_probs[category2index[next_day]] > three_day_model_next_day_probs[category2index[next_day]]):
                one_count += 2
            else:
                three_count += 2
        else:
            if (one_day_model_next_day_probs[category2index[next_day]] > two_day_model_next_day_probs[category2index[next_day]]):
                one_count += 1
            else:
                three_count += 1

    if one_count > two_count:
        one_wins += 1
    else:
        three_wins += 1
    #print('one count is: ' + str(one_count) + ' and three_count is ' + str(three_count))
    
print('One day model won ' + str(one_wins) + ' times')
print('Three day model won ' + str(three_wins) + ' times')

One day model won 0 times
Three day model won 10000 times
